# AI Multitask Classifiers: From Objects to Emotions

## Introduction
This project showcases various AI classifiers including object detection, face detection, character recognition, and mood classification, using OpenCV, TensorFlow, PyTorch, and Tesseract.

## Setup
To start, install the required libraries:

```python
!pip install numpy opencv-python-headless tensorflow torch pytesseract matplotlib
```

## Import Libraries

```python
import numpy as np
import cv2
import tensorflow as tf
import torch
import pytesseract
import matplotlib.pyplot as plt
from PIL import Image
```

## Load Models

### Object Detection (YOLOv3)

```python
yolo_model = cv2.dnn.readNetFromDarknet('path/to/yolov3.cfg', 'path/to/yolov3.weights')
yolo_model.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
yolo_model.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

with open("path/to/coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
```

### Face Detection (TensorFlow)

```python
tf_model = tf.keras.models.load_model('path/to/face_detection_model.h5')
```

### Mood Classification (PyTorch)

```python
torch_model = torch.load('path/to/mood_classification_model.pth')
torch_model.eval()
```

## Running Classifications

### Object Detection with YOLO

```python
def detect_objects_yolo(image_path, model, classes):
    image = cv2.imread(image_path)
    blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416), swapRB=True, crop=False)
    model.setInput(blob)
    
    layer_names = model.getLayerNames()
    output_layers = [layer_names[i[0] - 1] for i in model.getUnconnectedOutLayers()]
    
    outputs = model.forward(output_layers)
    
    height, width = image.shape[:2]
    boxes, confidences, class_ids = [], [], []
    
    for output in outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                box = detection[0:4] * np.array([width, height, width, height])
                centerX, centerY, w, h = box.astype("int")
                x = int(centerX - (w / 2))
                y = int(centerY - (h / 2))
                boxes.append([x, y, int(w), int(h)])
                confidences.append(float(confidence))
                class_ids.append(class_id)
                
    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    results = [(boxes[i], confidences[i], classes[class_ids[i]]) for i in indices.flatten()]
    return results

yolo_results = detect_objects_yolo('path/to/object/image.jpg', yolo_model, classes)
print("YOLO Object Detection Results:", yolo_results)
```

### Face Detection with TensorFlow

```python
def detect_faces_tf(image_path, model):
    image = Image.open(image_path)
    image = image.resize((224, 224))
    image_array = np.array(image) / 255.0
    image_array = np.expand_dims(image_array, axis=0)
    
    predictions = model.predict(image_array)
    return predictions

tf_results = detect_faces_tf('path/to/face/image.jpg', tf_model)
print("TensorFlow Face Detection Results:", tf_results)
```

### Mood Classification with PyTorch

```python
def classify_mood_torch(image_path, model):
    image = Image.open(image_path)
    transform = torch.nn.Sequential(
        torch.transforms.Resize((224, 224)),
        torch.transforms.ToTensor(),
        torch.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    )
    
    image_tensor = transform(image).unsqueeze(0)
    with torch.no_grad():
        output = model(image_tensor)
    _, predicted = torch.max(output, 1)
    mood = 'Happy' if predicted.item() == 1 else 'Sad'
    return mood

torch_results = classify_mood_torch('path/to/mood/image.jpg', torch_model)
print("PyTorch Mood Classification Results:", torch_results)
```

## Results and Analysis

### Displaying YOLO Object Detection Results

```python
def display_yolo_results(image_path, results):
    image = cv2.imread(image_path)
    for (box, confidence, label) in results:
        x, y, w, h = box
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(image, f"{label}: {confidence:.2f}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.show()

display_yolo_results('path/to/object/image.jpg', yolo_results)
```

### Displaying TensorFlow Face Detection Results

```python
print("TensorFlow Face Detection Results:", tf_results)
```

### Displaying PyTorch Mood Classification Results

```python
print("Mood Classification Result:", torch_results)
```

## Additional Classifiers

### Character Recognition (OCR)

```python
def recognize_text(image_path):
    image = cv2.imread(image_path)
    text = pytesseract.image_to_string(image)
    return text

ocr_result = recognize_text('path/to/text/image.jpg')
print("OCR Result:", ocr_result)
```

### Speech Recognition

```python
import speech_recognition as sr

def recognize_speech(audio_path):
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio_path) as source:
        audio = recognizer.record(source)
    text = recognizer.recognize_google(audio)
    return text

speech_result = recognize_speech('path/to/audio/file.wav')
print("Speech Recognition Result:", speech_result)
```

### Sentiment Analysis

```python
from transformers import pipeline

sentiment_analyzer = pipeline("sentiment-analysis")
sentiment_result = sentiment_analyzer("This is a great project!")
print("Sentiment Analysis Result:", sentiment_result)
```

## Conclusion
This project demonstrates using multiple classifiers for tasks such as object detection, face detection, mood classification, OCR, and sentiment analysis.